# IS318 - Machine Learning

## TP1 - Linear regression

The goal of this TP is to experiment with linear regression and polynomial linear regression.

First, we will work **without** the use of external libraries (such as `scikit-learn`).

In [23]:
import numpy as np
import matplotlib.pyplot as plt

### 1. Dataset

In [ ]:
# Let us generate data points from a relatively complicated function
N = 100
alpha, sigma = 2., 3.
X = np.linspace(-3, 3, num=N)
y = X + (alpha * np.sin((2. * np.pi * X) / sigma) * np.exp(-(X ** 2) / (sigma ** 2)))
# Add some random noise
rng = np.random.default_rng(42)
y_noisy = y + rng.standard_normal(N) * 2.
# Show the data points
plt.plot(X, y_noisy, 'o')
plt.plot(X, y)
plt.tight_layout()

**(Question)** Shuffle and split the dataset into training (75%) and validation (25%) sets. Store the results into variables `X_train`, `y_train`, `X_valid`, `y_valid`.

*Hint:* you can use `rng.permutation` to generate a random permutation.

In [ ]:
# print(np.shape(X), np.shape(y_noisy))
train_threshold = int(0.75*len(X))
# print(train_threshold)

stack = np.array([X,y_noisy,y])
shuffeled_stack = rng.permutation(stack, axis=1)
# print(stack)
# print("="*20)
# print(shuffeled_stack)

X_train = shuffeled_stack[0][:train_threshold]
y_train = shuffeled_stack[1][:train_threshold]

X_valid = shuffeled_stack[0][train_threshold:]
y_valid = shuffeled_stack[1][train_threshold:]

print(np.shape(X_train),np.shape(y_train))
print(np.shape(X_valid),np.shape(y_valid))

In [26]:
assert X_train.shape == (75,)
assert y_train.shape == (75,)
assert X_valid.shape == (25,)
assert y_valid.shape == (25,)
assert np.any(X_valid != X[75:]) # points should be shuffled

### 2. Linear regression in 1D

Recall the 1D linear regression model, where we search for parameters $w_0, w_1$ that will satisfy $y_i = w_0 + w_1 x_i$ (for all $i$ in the training set).

To simplify calculations, we usually set $\textbf{w} = [w_0, w_1]^T$ and $\textbf{x}_i = [1, x_i]^T$.

Then we have $y_i = \textbf{w}^T \textbf{x}_i $.

**(Question)** Add a column with ones to the points in `X_train` and `X_valid`. Store the result in new variables `X_train_ones` and `X_valid_ones`:

In [27]:
# print(X_train)
X_train_ones = np.transpose(np.array([np.ones(len(X_train)),X_train]))
X_valid_ones = np.transpose(np.array([np.ones(len(X_valid)),X_valid]))
# print(X_valid_ones)

**(Question)** Use normal equations to find the parameters that minimize the mean squared error on the training set.

In [ ]:


# The best weights are (X^TX)^{-1}X^Ty
w = np.array(np.shape(X_train_ones))
inv = np.linalg.inv(np.dot(np.transpose(X_train_ones),X_train_ones))
weight = np.dot(inv, np.transpose(X_train_ones))
w = np.dot(weight, y_train)

print(w)

In [29]:
assert w.shape == (2,)

**(Question)** Is it always possible to apply this solution? Can you think of example situations where it might not work?

This solution is only applicable in the case of linear regression and only if the matrix X^TX is inversable which means that all the features in the dataset are uncorelated and unrelated.

**(Question)** Plot the fitted line on top of the data points. Explain the result. (complete the code and comment in the cell below)

In [ ]:
print(np.shape(X_train))
plt.plot(X, [w[0]+x*w[1] for x in X], 'r--')
plt.plot(X, y_noisy, 'bo')
plt.plot(X, y, 'g')
plt.tight_layout()

**(Question)** Plot the loss function landscape. What can you conclude from this visualization? (complete the code and comment in the cell below)

*Hint: the loss landscape can be represented by a 2D map (for example of size 100x100) where in each coordinate $(w_0, w_1)$ the value is mean squared error for these parameters. You can use `plt.contourf` to visualize the result.*

In [ ]:
def MSE(y_hat, y):
    return np.sum((y - y_hat)**2)/len(y_hat)


grid_size = 100

_x = np.linspace(start=-5,stop=5,num=grid_size)
_y = np.linspace(start=-5,stop=5,num=grid_size)
_z = np.random.rand(grid_size, grid_size)
# print(x)
# _z = np.reshape(_z, (-1, grid_size))
_x, _y = np.meshgrid(_x, _y)

print(np.shape(_x),np.shape(_y),np.shape(_z))

for i in range(grid_size):
    for j in range(grid_size):
        _z[i,j] = MSE([_x[i,j]+_y[i,j]*x for x in X_train], y_train)
    

fig = plt.figure()
ax1 = plt.contourf(_x, _y, _z)

plt.show()

### 3. Polynomial linear regression

We move on the polynomial linear regression model with degree $D$, where the relationship between $y_i$ and $x_i$ is
$ y_i = w_0 + w_1x_i + w_2x_i^2 + \ldots + w_D x_i^D $

With $\textbf{w} = [w_0, \ldots, w_D]^T$ and $\textbf{x}_i = [1, x_i, x_i^2, \ldots, x_i^D]^T$, we have $y_i = \textbf{w}^T \textbf{x}_i $.

$D \geq 1$ is an hyperparameter of the model.

**(Question)** Complete the following `PolynomialRegression` class

In [32]:
class PolynomialRegression():
    def __init__(self, D=1):
        assert D >=1
        self.D = D
        
    def fit(self, X, y):
        '''Apply polynomial linear regression to fit `X` to `y`.
        The result should be stored in an attribute `w`.'''
        # print(X.shape)
        X_ones = self.make_poly(X)
        # print(y.shape)
        self.w = np.empty((self.D, 1))
        inv = np.linalg.inv(np.dot(np.transpose(X_ones),X_ones))
        weight = np.dot(inv, np.transpose(X_ones))
        self.w = np.dot(weight, y)
        # print(f"The shape of w is {self.w.shape}")
        
    def predict(self, X):
        '''Assuming the model has already been fit, return
        predicted `y` values for given `X`.'''
        X_ones = self.make_poly(X)
        # print(np.shape(self.w), np.shape(X_ones))
        y_pred = np.empty((len(X_ones),))
        for i in range(len(X_ones)):
            # print(f"Dot product shapes {self.w.shape}x{X_ones[i].shape}")
            # print(f"=={np.dot(np.transpose(self.w), X_ones[i]).shape}")
            y_pred[i] = np.dot(np.transpose(self.w), X_ones[i])
            # print(y_pred)
        return y_pred

    def make_poly(self, X):
        '''Augment a dataset of 1D points (vector of size N) to its
        data matrix in polynomial form, including the zero column 
        (matrix of size N x D+1). Return the data matrix.'''
        assert X.ndim == 1
        X_ones = np.empty((len(X), self.D))
        T = np.transpose(X_ones)
        for d in range(self.D):
            # print(f"=={d}")
            # print(X[:3], np.power(X[:3], d))
            T[d] =  np.power(X, d)
        # print(T)
        return np.transpose(T)

**(Question)** Implement the mean squared error function to measure the quality of predictions.

In [33]:
def mean_squared_error(y_true, y_pred):
    '''Return the mean squared error between `y_true` and `y_pred`.'''
    # print(f"{y_true.shape} == {y_pred.shape}")
    assert y_true.shape == y_pred.shape
    return np.sum((y_true - y_pred)**2)/len(y_pred)

In [34]:
a, b = np.random.randn(10), np.random.randn(10)
assert mean_squared_error(a, b) >= 0.
assert mean_squared_error(a, a) == 0.

**(Question)** Apply the polynomial regression model with $D=5$. Compute and print the mean squared error for the training and validation sets.

In [ ]:
poly_reg_5 = PolynomialRegression(5)
poly_reg_5.fit(X_train, y_train)
# print(np.shape(y_train), np.shape(poly_reg_5.predict(X_train)))
print(mean_squared_error(y_train, poly_reg_5.predict(X_train)))
print(mean_squared_error(y_valid, poly_reg_5.predict(X_valid)))

**(Question)** Plot the fitted polynomial curve on top of the data points. Explain the result. (complete the code and comment in the cell below)

In [ ]:
print(np.shape(X_train))
plt.plot(X, poly_reg_5.predict(X), 'r--')
plt.plot(X, y_noisy, 'bo')
plt.plot(X, y, 'g')
plt.tight_layout()

**(Question)** Using the validation set, implement a simple model selection strategy to optimize hyperparameter $D$ and print this value. For this question, you should limit the search to $D \in [1, 15]$.
To visualize potential underfitting and overfitting effects, plot the evolution of the error on the training and the validation sets for the different values of $D$.

In [ ]:
D = np.arange(start=1,stop=16)
train_mse = []
valid_mse = []
for d in D:
    poly_reg = PolynomialRegression(d)
    poly_reg.fit(X_train, y_train)
    train_mse.append(mean_squared_error(y_train, poly_reg.predict(X_train)))
    valid_mse.append(mean_squared_error(y_valid, poly_reg.predict(X_valid)))


plt.plot(D, train_mse, 'bo--')
plt.plot(D, valid_mse, 'ro-')

**(Question)** Plot the fitted polynomial curve of the best model on top of the data points. Comment the results.

In [ ]:
print(np.shape(X_train))
poly_reg_9 = PolynomialRegression(9)
poly_reg_9.fit(X_train, y_train)

plt.plot(X, poly_reg_9.predict(X), 'r--')
plt.plot(X, y_noisy, 'bo')
plt.plot(X, y, 'g')
plt.tight_layout()

### 4. Regularized polynomial regression

Now, we want to implement polynomial regression with *weight decay* regularization:
$\hat{L}(\textbf{w}) = \frac{1}{N} \lVert \textbf{X} \textbf{w} - \textbf{y} \rVert^2 + \lambda \lVert\textbf{w}\rVert^2$

Here, $\lambda \geq 0$ is another hyperparameter of our model.

**(Question)** Complete the following `RegularizedPolynomialRegression` class.

In [39]:
class RegularizedPolynomialRegression():
    def __init__(self, D=1, lmbda=1.):
        assert D >=1 and lmbda >= 0.
        self.D = D
        self.lmbda = lmbda
        
    def fit(self, X, y):
        '''Apply polynomial linear regression to fit `X` to `y`.
        The result should be stored in an attribute `w`.'''
        X_ones = self.make_poly(X)
        # print(y.shape)
        self.w = np.empty((self.D, 1))
        decay = self.lmbda*np.identity(self.D)
        inv = np.linalg.inv(np.add(decay, np.dot(np.transpose(X_ones),X_ones)))
        weight = np.dot(inv, np.transpose(X_ones))
        self.w = np.dot(weight, y)
        
    def predict(self, X):
        '''Assuming the model has already been fit, return
        predicted `y` values for given `X`.'''
        X_ones = self.make_poly(X)
        # print(np.shape(self.w), np.shape(X_ones))
        y_pred = np.empty((len(X_ones),))
        for i in range(len(X_ones)):
            # print(f"Dot product shapes {self.w.shape}x{X_ones[i].shape}")
            # print(f"=={np.dot(np.transpose(self.w), X_ones[i]).shape}")
            y_pred[i] = np.dot(np.transpose(self.w), X_ones[i])
            # print(y_pred)
        return y_pred

    def make_poly(self, X):
        '''Augment a dataset of 1D points (vector of size N) to its
        data matrix in polynomial form, including the zero column 
        (matrix of size N x D+1). Return the data matrix.'''
        assert X.ndim == 1
        X_ones = np.empty((len(X), self.D))
        T = np.transpose(X_ones)
        for d in range(self.D):
            T[d] =  np.power(X, d)
        return np.transpose(T)

**(Question)** Apply regularized linear regression and play around with hyperparameters $D$ and $\lambda$. Plot the result.

In [ ]:
print(np.shape(X_train))
poly_reg_9 = RegularizedPolynomialRegression(11,2)
poly_reg_9.fit(X_train, y_train)

plt.plot(X, poly_reg_9.predict(X), 'r--')
plt.plot(X, y_noisy, 'bo')
plt.plot(X, y, 'g')
plt.tight_layout()

In [ ]:
lamb = np.arange(start=0,stop=6)
train_mse = []
valid_mse = []
for l in lamb:
    poly_reg = RegularizedPolynomialRegression(11,l)
    poly_reg.fit(X_train, y_train)
    train_mse.append(mean_squared_error(y_train, poly_reg.predict(X_train)))
    valid_mse.append(mean_squared_error(y_valid, poly_reg.predict(X_valid)))


plt.plot(train_mse, 'bo--', label="train")
plt.plot(valid_mse, 'ro-', label="valid")
plt.legend()

In [ ]:
D = np.arange(start=1,stop=16)


train_mse = []
valid_mse = []
for d in D:
    poly_reg = PolynomialRegression(d)
    poly_reg.fit(X_train, y_train)
    train_mse.append(mean_squared_error(y_train, poly_reg.predict(X_train)))
    valid_mse.append(mean_squared_error(y_valid, poly_reg.predict(X_valid)))


plt.plot(D, train_mse, 'ro--', label="train")
plt.plot(D, valid_mse, 'ro-', label="valid")


train_mse = []
valid_mse = []
for d in D:
    poly_reg = RegularizedPolynomialRegression(d,2)
    poly_reg.fit(X_train, y_train)
    train_mse.append(mean_squared_error(y_train, poly_reg.predict(X_train)))
    valid_mse.append(mean_squared_error(y_valid, poly_reg.predict(X_valid)))


plt.plot(D, train_mse, 'go--', label="train_decay")
plt.plot(D, valid_mse, 'go-', label="valid_decay")
plt.legend()

### 5. Comparison with `sklearn`

In [43]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge

**(Question)** Use the `sklearn` classes imported above to apply polynomial regression on our toy dataset. Compare the results with your implementation and comment.

In [ ]:
# PolynomialFeatures((0,11))

linear_reg = LinearRegression()

linear_reg.fit(X_train, y_train)
print(f"shapes{X_train.shape} and {y_train.shape}")
print(linear_reg.score(X_valid, y_valid))

y_pred = linear_reg.predict(X_valid) 
plt.scatter(X_valid, y_valid, color ='b') 
plt.plot(X_valid, y_pred, color ='k') 
  
plt.show() 